# Motion generation from checkpoints

In [ ]:
// for local development
%install-location /notebooks/language2motion.gt/swift-install
%install-swiftpm-flags -c release
%install '.package(path: "/notebooks/language2motion.gt")' Datasets TranslationModels TextModels ModelSupport SummaryWriter LangMotionModels Checkpoints

In [ ]:
import TensorFlow
import TextModels
import TranslationModels
import Foundation
import FoundationXML
import ModelSupport
import Datasets
import SummaryWriter
import LangMotionModels
import Checkpoints
import PythonKit

In [ ]:
let np  = Python.import("numpy")

In [ ]:
%include "EnableIPythonDisplay.swift"
IPythonDisplay.shell.enable_matplotlib("inline")

## Set training params

In [ ]:
let device = Device.defaultTFEager

In [ ]:
let maxTextSequenceLength =  20
let maxMotionLength =  100

In [ ]:
let datasetSize: DatasetSize = .multi_full
let batchSize = 150

In [ ]:
let dataURL = URL(fileURLWithPath: "/notebooks/language2motion.gt/data/")
let motionDatasetURL = dataURL.appendingPathComponent("motion_dataset_v3.10Hz.\(datasetSize.rawValue)plist")

In [ ]:
/// instantiate text processor
let vocabularyURL = dataURL.appendingPathComponent("vocab.txt")
let vocabulary: Vocabulary = try! Vocabulary(fromFile: vocabularyURL)
let tokenizer: Tokenizer = BERTTokenizer(vocabulary: vocabulary, caseSensitive: false, unknownToken: "[UNK]", maxTokenLength: nil)
let textProcessor = TextProcessor(vocabulary: vocabulary, tokenizer: tokenizer)

// model config
let config = LangMotionTransformerConfig(
    vocabSize: vocabulary.count,
    nbJoints: 47, // TODO: get value from dataset
    nbMixtures: 20,
    layerCount: 6,
    modelSize: 256,
    feedForwardSize: 1024,
    headCount: 8,
    dropoutProbability: 0.1,
    sentenceMaxPositionalLength: 100, 
    motionMaxPositionalLength: 500,
    doMotionDense: false
)

In [ ]:
print("\nLoading dataset...")

var dataset = try Lang2Motion(
    motionDatasetURL: motionDatasetURL,
    batchSize: batchSize,
    minMotionLength: 10,
    maxMotionLength: 100,
    trainTestSplit: 1.0,
    device: device
) { (motionSample: MotionSample) -> LangMotionBatch in    
    let sentence = textProcessor.preprocess(sentence: motionSample.annotations[0], maxTextSequenceLength: maxTextSequenceLength)
    let (motionPart, target) = LangMotionBatch.preprocessTargetMotion(sampleID: motionSample.sampleID, motion: motionSample.motion, maxMotionLength: maxMotionLength)
    let source = LangMotionBatch.Source(sentence: sentence, motionPart: motionPart)
    let singleBatch = LangMotionBatch(data: source,label: target)
    return singleBatch
}

print("Dataset acquired.")

## Helpers

In [ ]:
public func randomNumber(probabilities: [Double]) -> Int {
    // https://stackoverflow.com/questions/30309556/generate-random-numbers-with-a-given-distribution
    // Sum of all probabilities (so that we don't have to require that the sum is 1.0):
    let sum = probabilities.reduce(0, +)
    // Random number in the range 0.0 <= rnd < sum :
    let rnd = Double.random(in: 0.0 ..< sum)
    // Find the first interval of accumulated probabilities into which `rnd` falls:
    var accum = 0.0
    for (i, p) in probabilities.enumerated() {
        accum += p
        if rnd < accum {
            return i
        }
    }
    // This point might be reached due to floating point inaccuracies:
    return (probabilities.count - 1)
}

public func gaussian_pdf(sample: Tensor<Float>, means: Tensor<Float>, variances: Tensor<Float>) -> Tensor<Float> {
    // one-dim tensors
    assert(sample.shape.count == 1)
    assert(sample.shape == means.shape)
    assert(sample.shape == variances.shape)
    let a1 = sqrt(Float(2.0) * Float(np.pi)! * variances)
    let a2 = -(sample - means).squared()
    return Float(1.0) / a1 * exp(a2 / (2.0 * variances))
}

public func bernoulli_pdf(sample: Int, p: Float) -> Float {
    let fSample = Float(sample)
    return fSample * p + Float(1.0 - fSample) * (1.0 - p)
}

public class MotionDecoder2 {
    public static func performNormalMixtureSampling(preds: MixtureModelPreds, nb_joints: Int, nb_mixtures: Int, maxMotionLength: Int) -> (motion: Tensor<Float>, log_probs: [Float], done: Tensor<Int32>) {
        let TINY: Float = 1e-8
        let motionLength = preds.mixtureMeans.shape[1]

        var motion: Tensor<Float> = Tensor<Float>(zeros: [motionLength, nb_joints])
        var log_probs: [Float] = [Float](repeating:0.0, count: motionLength)
        var done: [Int32] = [Int32](repeating: 0, count: motionLength)

        let all_means = preds.mixtureMeans.squeezingShape(at: 0)
        let all_variances = preds.mixtureVars.squeezingShape(at: 0) + TINY
        let weights = preds.mixtureWeights.squeezingShape(at: 0)
        let stops = preds.stops[0, 0..., 0]

        /// Sample joint values.
        var samples = Tensor<Float>(zeros: [motionLength, nb_joints])
        var means = Tensor<Float>(zeros: [motionLength, nb_joints])
        var variances = Tensor<Float>(zeros: [motionLength, nb_joints])
        for width_idx in 0..<motionLength {
            // Decide which mixture to sample from
            let p = weights[width_idx].scalars.map { Double($0)}
            assert(p.count == nb_mixtures)
            let mixture_idx = randomNumber(probabilities: p) //np.random.choice(range(nb_mixtures), p=p)

            /// Sample from it.
            let start_idx = mixture_idx * nb_joints
            let m = all_means[width_idx, start_idx..<start_idx + nb_joints]
            let v = all_variances[width_idx, start_idx..<start_idx + nb_joints]
            assert(m.shape == [nb_joints])
            assert(m.shape == v.shape)
            // https://numpy.org/doc/stable/reference/random/generated/numpy.random.normal.html
            let s = np.random.normal(m.scalars, v.scalars)
            samples[width_idx, 0...] = Tensor<Float>(Array(s)!)
            means[width_idx, 0...] = m
            variances[width_idx, 0...] = v
        }

        for idx in 0..<motionLength {
            let sample = samples[idx]
            let stop: Float = stops[idx].scalar!
            // if done[idx] != 0 {
            //     continue
            // }
            motion[idx] = sample
            // https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.binomial.html
            let sampled_stop: Int = Int(np.random.binomial(n: 1, p: stop))!
            log_probs[idx] += log(gaussian_pdf(sample: sample, means: means[idx], variances: variances[idx])).sum().scalar!
            log_probs[idx] += log(bernoulli_pdf(sample: sampled_stop, p: stop))
            done[idx] = (sampled_stop == 0) ? 1 : 0
        }
        return (motion: motion, log_probs: log_probs, done: Tensor(done))
    }
    
    public static func greedyDecodeMotion(
        sentence: LangMotionBatch.Sentence, 
        startMotion: Tensor<Float>?,
        transformer: LangMotionTransformer, 
        nbJoints: Int, 
        nbMixtures: Int, 
        maxMotionLength: Int,
        memoryMultiplier: Float = 1.0
    ) -> Tensor<Float> {
        print("\nEncode:")
        print("======")
        let memory = transformer.encode(input: sentence) //* memoryMultiplier
        print("  memory.count: \(memory.shape)")     

        print("\nGenerate:")
        print("=========")

        // start with tensor for neutral motion frame        
        var ys: Tensor<Float> = LangMotionBatch.startMotionToken(nbJoints: nbJoints).expandingShape(at: 0)
        // or with supplied motion
//         if startMotion != nil {
//             ys = startMotion!.expandingShape(at:0)            
//         }

        print("ys.shape: \(ys.shape)")
        
        let maxMotionLength2 = maxMotionLength-ys.shape[1]+1
//         print("maxMotionLength2: \(maxMotionLength2)")
        
        for step in 0..<maxMotionLength2 {
//             print("step: \(step)")
            // prepare input
            // let motionPartMask = Tensor<Float>(LangMotionBatch.subsequentMask(size: ys.shape[1]))
            let motionPartFlag = Tensor<Int32>(repeating: 1, shape: [1, ys.shape[1]])
            let motionPartMask = LangMotionBatch.makeStandardMask(target: motionPartFlag, pad: 0)
            let motionPart = LangMotionBatch.MotionPart(motion: ys, mask: motionPartMask)

            // decode motion
            let dedoderOutput = transformer.decode(sourceMask: sentence.mask, motionPart: motionPart, memory: memory)
            
//             print("dedoderOutput")
            
            let mixtureModelInput = Tensor<Float>(concatenating: dedoderOutput.allOutputs, alongAxis: 2)
//             print("mixtureModelInput.shape: \(mixtureModelInput.shape)")
            let singlePreds = transformer.mixtureModel(mixtureModelInput[0...,-1].expandingShape(at: 0))

            
            // perform sampling
            let (sampledMotion, _, _) = MotionDecoder.performNormalMixtureSampling(
                preds: singlePreds, nb_joints: nbJoints, nb_mixtures: nbMixtures, maxMotionLength: maxMotionLength)
            
            // concatenate motion
            ys = Tensor(concatenating: [ys, sampledMotion.expandingShape(at: 0)], alongAxis: 1)        
        }
        return ys.squeezingShape(at:0)[1...]
    }    
}

In [ ]:
public struct SampleMotionClip {
    var sampleID: Int
    var start: Int = 0
    var length: Int = 1
}

public func getClippedMotionFrames(dataset: Lang2Motion, clipInfo: SampleMotionClip?) -> Tensor<Float>? {
    if clipInfo != nil {
    
    let ms: MotionSample = dataset.motionSamples.filter { $0.sampleID == clipInfo!.sampleID } [0]
    let clippedMotionFrames = ms.motion[clipInfo!.start..<clipInfo!.start+clipInfo!.length]
    return clippedMotionFrames
    } else {
        return nil
    }
}

In [ ]:
public func greedyDecodeMotion(dataset: Lang2Motion, model: LangMotionTransformer, sentence: String, leadingFrames: SampleMotionClip?, prefix: String = "prefix", saveMotion: Bool = true, memoryMultiplier: Float = 0.0, motionsURL: URL?) {
    let startMotion: Tensor<Float>? = getClippedMotionFrames(dataset: dataset, clipInfo: leadingFrames)
    var leadingFramesStr = "0"
    if startMotion != nil {
        leadingFramesStr = "\(startMotion!.shape[0])"
    }
    // TODO: incorporate done/stop signal
    Context.local.learningPhase = .inference
    print("\ngreedyDecodeMotion(sentence: \"\(sentence)\")")

    let processedSentence = textProcessor.preprocess(sentence: sentence, maxTextSequenceLength: maxTextSequenceLength)
    processedSentence.printSentence()

    let decodedMotion = MotionDecoder2.greedyDecodeMotion(
        sentence: processedSentence, 
        startMotion: startMotion,
        transformer: model, 
        nbJoints: config.nbJoints, 
        nbMixtures: config.nbMixtures, 
        maxMotionLength: maxMotionLength,
        memoryMultiplier: memoryMultiplier
    )
    print("  decodedMotion: min: \(decodedMotion.min()), max: \(decodedMotion.max())")
    let descaledMotion = dataset.scaler.inverse_transform(decodedMotion)
    print("  descaledMotion.shape: \(descaledMotion.shape)")
    print("  descaledMotion: min: \(descaledMotion.min()), max: \(descaledMotion.max())")
    var imageURL: URL? = nil
    
    if !saveMotion { imageURL = nil } else {
        imageURL = motionsURL!.appendingPathComponent("\(prefix).png")
    }
    // use joint groupping
    let grouppedJointsMotion = MotionSample.grouppedJoints(motion: descaledMotion, jointNames: dataset.motionSamples[0].jointNames)
    motionToImg(url: imageURL, motion: grouppedJointsMotion, motionFlag: nil, padTo: maxMotionLength, descr: "\(sentence), LF: \(leadingFramesStr)", cmapRange: 1.0)

    if saveMotion {
        print("Saved image: \(imageURL!.path)")
        let jointNames = dataset.motionSamples[0].jointNames
        let mmmXMLDoc = MMMWriter.getMMMXMLDoc(jointNames: jointNames, motion: descaledMotion)
        let mmmURL = motionsURL!.appendingPathComponent("\(prefix).mmm.xml")
        try! mmmXMLDoc.xmlData(options: XMLNode.Options.nodePrettyPrint).write(to: mmmURL)
        print("Saved motion: \(mmmURL.path)")
    }
}

In [ ]:
func showMotionSample(_ motionSample: MotionSample) {
    let motion = motionSample.motion
    let descaledMotion = dataset.scaler.inverse_transform(motion)
    let sentence = "sample_id=\(motionSample.sampleID), ann=\(motionSample.annotations[0])"

    print("motion: min: \(motion.min()), max: \(motion.max())")
    print("descaledMotion.shape: \(descaledMotion.shape)")
    print("descaledMotion: min: \(descaledMotion.min()), max: \(descaledMotion.max())")

    // use joint groupping
    let jointNames = dataset.motionSamples[0].jointNames
    let grouppedJointsMotion = MotionSample.grouppedJoints(motion: descaledMotion, jointNames: dataset.motionSamples[0].jointNames)
    motionToImg(url: nil, motion: grouppedJointsMotion, motionFlag: nil, padTo: maxMotionLength, descr: sentence, cmapRange: 1.0)
}

In [ ]:
func showMotion(motion: Tensor<Float>) {
    let descaledMotion = dataset.scaler.inverse_transform(motion)
    let grouppedJointsMotion = MotionSample.grouppedJoints(motion: descaledMotion, jointNames: dataset.motionSamples[0].jointNames)
    motionToImg(url: nil, motion: grouppedJointsMotion, motionFlag: nil, padTo: maxMotionLength, descr: "", cmapRange: 1.0)
}

In [ ]:
func saveMotionToMMM(motion: Tensor<Float>, mmmURL: URL) {
    let descaledMotion = dataset.scaler.inverse_transform(motion)
    let jointNames = dataset.motionSamples[0].jointNames
    let mmmXMLDoc = MMMWriter.getMMMXMLDoc(jointNames: jointNames, motion: descaledMotion)
    try! mmmXMLDoc.xmlData(options: XMLNode.Options.nodePrettyPrint).write(to: mmmURL)
    print("Saved motion: \(mmmURL.path)")
}

## Load model checkpoint

In [ ]:
let runName = "run_34"
let epoch = 1

In [ ]:
let runURL = dataURL.appendingPathComponent("runs/Lang2motion/\(runName)", isDirectory: true)
let checkpointURL = runURL.appendingPathComponent("checkpoints", isDirectory: true)
let motionsURL = runURL.appendingPathComponent("generated_motions", isDirectory: true)
try! FileManager().createDirectory(at: motionsURL, withIntermediateDirectories: true)

let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.e\(epoch)")
// let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.final")

## Decode using leading motion frames

### Find suitable motion sample

In [ ]:
let annotations = dataset.langRecs
annotations.count

In [ ]:
let search = "guitar"
let filteredAnns = annotations.filter { $0.text.contains(search) }
print(filteredAnns.count)
let startIdx = 0
filteredAnns[startIdx..<startIdx+10].map { (sampleID: $0.sampleID, ann: $0.text) }

### Select motion sample

In [ ]:
let selAnn = filteredAnns[3]
let selSampleInfo = (sampleID: selAnn.sampleID, text: selAnn.text, length: selAnn.motionSample.motion.shape[0])

print("Selected motion sample")
print(selSampleInfo)
showMotionSample(selAnn.motionSample)
saveMotionToMMM(motion: selAnn.motionSample.motion, mmmURL: motionsURL.appendingPathComponent("sample.mmm.xml"))

### Clip motion

In [ ]:
let clipInfo = SampleMotionClip(sampleID: selSampleInfo.sampleID, start: 14, length: 10)

In [ ]:
let clippedMotionFrames: Tensor<Float>? = getClippedMotionFrames(dataset: dataset, clipInfo: clipInfo)
print("\n**** \(clipInfo) ****\n")
print("Actual length: \(clippedMotionFrames!.shape[0])")
print("clippedMotionFrames: min: \(clippedMotionFrames!.min()), max: \(clippedMotionFrames!.max())")
showMotion(motion: clippedMotionFrames!)
saveMotionToMMM(motion: clippedMotionFrames!, mmmURL: motionsURL.appendingPathComponent("clip.mmm.xml"))

## Load model checkpoint

In [ ]:
let runName = "run_34"
let epoch = 20

In [ ]:
let runURL = dataURL.appendingPathComponent("runs/Lang2motion/\(runName)", isDirectory: true)
let checkpointURL = runURL.appendingPathComponent("checkpoints", isDirectory: true)
let motionsURL = runURL.appendingPathComponent("generated_motions", isDirectory: true)
try! FileManager().createDirectory(at: motionsURL, withIntermediateDirectories: true)

let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.e\(epoch)")
// let model = LangMotionTransformer(checkpoint: checkpointURL, config: config, name: "model.final")

### Generate motion

In [ ]:
var genNum = 1

In [ ]:
var s: String = ""
var lf: SampleMotionClip?

In [ ]:
s = "A person is walking forwards five steps."
// s = "A person is walking forwards."
// lf = SampleMotionClip(sampleID: 1, start: 26, length: 2)
lf = nil

// s = "A person plays the guitar."
// lf = SampleMotionClip(sampleID: 1417, start: 14, length: 10)

// s = "The human plays air guitar and sways ans stands still."
// s = "The human walks in the straight line."
// s = "Someone is jogging."

// s = "a person waves with his both arms"
// s = "a person is waving his hand."
// s = "A person runs."
// s = "A person kneels down."
// s = "A human walking backwards"
// s = "A person walks 4 steps forward."

In [ ]:
greedyDecodeMotion(dataset: dataset, model: model, sentence: s, leadingFrames: lf, 
    prefix: "epoch_\(epoch)_motion_\(genNum)", 
    saveMotion: true,  memoryMultiplier: 1.0, motionsURL: motionsURL
)
genNum += 1